# 13.2.3 TPU Training

TPUs are available in Colab. Here is a template to get you started.

#### Note

Sadly, given the huge popularity of the service, Google has introduced paid tiers and reduce (eliminated?) free TPU access. I do remember being able to experiment with this, but of late I have not been able to connect to a TPU runtime any longer... It might be that the free version of Colab no longer offers TPUs, even if the option is there. Let me know if you are able to connect!

In [ ]:
import tensorflow as tf

# connect to the TPU cluster
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
print("Device:", tpu.master())

In [ ]:
# we need to tell TensorFlow to use the hardware, using a 'strategy'
strategy = tf.distribute.TPUStrategy(tpu)
print(f"Number of replicas: {strategy.num_replicas_in_sync}") # a TPU has 8 cores

(Listing 13.4)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# functional encapsulation: everything inside the function
# will be created following the strategy
def build_model(input_size):
    inputs = keras.Input((input_size, input_size, 3))
    x = keras.applications.resnet.preprocess_input(inputs)
    x = keras.applications.resnet.ResNet50(
        weights=None, include_top=False, pooling="max"
    )(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# we create our model in the 'scope' of the strategy
with strategy.scope():
    model = build_model(input_size=32)  

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

model.fit(
    x_train,
    y_train, 
    epochs=10,
    batch_size=1024 # notice the large batch size, TPUs have a large capacity!
)